In [1]:
%pwd

'c:\\Users\\Ayush\\OneDrive\\Desktop\\Coccidiosis-chicken-disease-classification\\research'

In [2]:
import os
os.chdir("../")

In [3]:
%pwd

'c:\\Users\\Ayush\\OneDrive\\Desktop\\Coccidiosis-chicken-disease-classification'

### 4. Update the entity

In [4]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class PrepareCallbacksConfig:
    root_dir: Path
    tensorboard_root_log_dir: Path
    checkpoint_model_filepath: Path

### 5. Update the configuration manager in src config

In [5]:
from cnnClassifier.constants import *
from cnnClassifier.utils.common import read_yaml, create_directories

In [6]:
class ConfigurationManager:
    def __init__(self, 
                 config_filepath=CONFIG_FILE_PATH, 
                 params_filepath=PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directories([self.config.artifacts_root])

    def get_prepare_callback_config(self) -> PrepareCallbacksConfig:
        config = self.config.prepare_callbacks
        model_ckpt_dir = os.path.dirname(config.checkpoint_model_filepath)
        
        create_directories([
            Path(model_ckpt_dir),
            Path(config.tensorboard_root_log_dir)
        ])
        
        prepare_callback_config = PrepareCallbacksConfig(
            root_dir=Path(config.root_dir),
            tensorboard_root_log_dir=Path(config.tensorboard_root_log_dir),
            checkpoint_model_filepath=Path(config.checkpoint_model_filepath)
        )
        
        return prepare_callback_config

### 6. Update the components

In [7]:
import os
import urllib.request as request
from zipfile import ZipFile
import tensorflow as tf
import time

In [ ]:
class PrepareCallback:
    def __init__(self, config: PrepareCallbacksConfig):
        self.config = config

    @property
    def _create_tb_callbacks(self):
        timestamp = time.strftime("%Y-%m-%d-%H-%M-%S")
        tb_runtime_log_dir = os.path.join(
            self.config.tensorboard_root_log_dir, f"tb_logs_at_{timestamp}")
        return tf.keras.callbacks.TensorBoard(log_dir=tb_runtime_log_dir)
    
    @property
    def _create_ckpt_callbacks(self):
        return tf.keras.callbacks.ModelCheckpoint(
            filepath=self.config.checkpoint_model_filepath,
            save_best_only=True
        )
    
    def get_tb_ckpt_callbacks(self):
        return [
            self._create_tb_callbacks,
            self._create_ckpt_callbacks
        ]

### 7. Prepare and update the pipeline

In [9]:
try:
    config = ConfigurationManager()
    prepare_callbacks_config = config.get_prepare_callback_config()
    prepare_callbacks = PrepareCallback(config=prepare_callbacks_config)
    tb_ckpt_callbacks = prepare_callbacks.get_tb_ckpt_callbacks()

except Exception as e:
    raise e

[2026-01-28 15:41:38,869: INFO: common: YAML file: config\config.yaml loaded successfully]
[2026-01-28 15:41:38,871: INFO: common: YAML file: params.yaml loaded successfully]
[2026-01-28 15:41:38,873: INFO: common: Directory created at: artifacts]
[2026-01-28 15:41:38,875: INFO: common: Directory created at: artifacts\prepare_callbacks\checkpoints_dir]
[2026-01-28 15:41:38,876: INFO: common: Directory created at: artifacts\prepare_callbacks\tensorboard_logs_dir]


In [11]:
#This is a sample code to generate timestamped folder name

import time
timestamp = time.strftime("%Y-%m-%d-%H-%M-%S")
f"tb_logs_at_{timestamp}"

'tb_logs_at_2026-01-28-15-43-50'